# dbachecks - Configurations

You can use dbachecks on the command line as we have seen in the previous notebook but you will not always want to have the default settings for the checks

You can alter any of the configurations using the `Set-DbcConfig` command.

## Invalid Database Owners

You may want to check that databases are not owned by certain accounts.

By default the `InvalidDatabaseOwner` checks that the databases are not owned by the sa account. When we run this check against the instance we pass all of the checks


In [2]:
Invoke-DbcCheck -SqlInstance SQL2016N1 -Check InvalidDatabaseOwner

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on SQL2016N1
      [+] Database AdventureWorksDW2014 - owner THEBEARD\Rob should Not be in this list ( sa ) on SQL2016N1 3ms
      [+] Database BeardedAg_db - owner THEBEARD\Rob should Not be in this list ( sa ) on SQL2016N1 2ms
      [+] Database DAC - owner THEBEARD\Rob should Not be in this list ( sa ) on SQL2016N1 1ms
      [+] Database database - owner THEBEARD\Rob should Not be in this list ( sa ) on SQL2016N1 1ms
      [+] Database DBA-Admin - owner THEBEARD\Rob should Not be in this list ( sa ) on SQL20

If our organistation requirements are that no databases should be owned by members of the DBA Team user accounts, we can alter the configuration to check for that. 

To find out which configuration we need to change we can use `Get-DbcCheck`

In [14]:
Get-DbcCheck -Pattern InvalidDatabaseOwner | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : InvalidDatabaseOwner
AllTags     : InvalidDatabaseOwner, Medium, Database
Config      : app.sqlinstance policy.invaliddbowner.name policy.invaliddbowner.excludedb 
Description : Tests that the Database Owner is NOT in the specified (default blank) list





We can see that there are 3 configurations that this check will use
- app.sqlinstance
- policy.invaliddbowner.name 
- policy.invaliddbowner.excludedb 

We can get further information about the checks with `Get-DbcConfig`

In [18]:
Get-DbcConfig -Name policy.invaliddbowner.name | Format-List



Name        : policy.invaliddbowner.name
Value       : sa
Description : The database owner account should not be this user





We can change the value of that configuration item using `Set-DbcConfig`. We will set it to the members of the DBA Team AD Group

In [19]:
$DBA_Accounts =  (Get-ADGroup -Identity 'CN=SQL_DBAs_The_Cool_Ones,OU=TheBeardGroups,OU=TheBeard,DC=TheBeard,DC=local' |Get-ADGroupMember).ForEach{"THEBEARD\$($Psitem.SamAccountName)"} 
Set-DbcConfig -Name policy.invaliddbowner.name -Value $DBA_Accounts


Name                       Value                                                                Description            
----                       -----                                                                -----------            
policy.invaliddbowner.name {THEBEARD\Rob, THEBEARD\DBA, THEBEARD\wdurkin, THEBEARD\gsartori...} The database owner a...




Now that we have set the configuration, we can run the check again

In [20]:
Invoke-DbcCheck -SqlInstance SQL2016N1 -Check InvalidDatabaseOwner

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on SQL2016N1
      [-] Database AdventureWorksDW2014 - owner THEBEARD\Rob should Not be in this list ( THEBEARD\Rob, THEBEARD\DBA, THEBEARD\wdurkin, THEBEARD\gsartori, THEBEARD\akamman, THEBEARD\clemaire, THEBEARD\smelton, THEBEARD\csilva, THEBEARD\fatherjack, THEBEARD\alevy, THEBEARD\jamrtin ) on SQL2016N1 76ms
        Expected collection @('THEBEARD\Rob', 'THEBEARD\DBA', 'THEBEARD\wdurkin', 'THEBEARD\gsartori', 'THEBEARD\akamman', 'THEBEARD\clemaire', 'THEBEARD\smelton', 'THEBEARD\csilva', 'THEBEARD\fatherjac

Ah

We had better have a word with that fellow Rob and alter all of those database owners!! 

(We could do that with `Get-DbaDatabase -SqlInstance sql2016n1 -ExcludeSystem -Owner THEBEARD\Rob | Set-DbaDbOwner -TargetLogin THEBEARD\DatabaseOwner`)

This is how you can alter a check to test what you want by setting a configuration

## Ensuring Databases exist

You might want to test that a particular database exists on your instance. This could be your DBA database or perhaps you have a check for your companies software system that you want to run on a clients estate and your software requires certain databases to exist.

You can run the default `DatabaseExists` which will check that the system databases exist

In [1]:
Invoke-DbcCheck -SqlInstance SQL2016N1 -Check DatabaseExists

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on SQL2016N1
      [+] Database master should exist on  506ms
      [+] Database msdb should exist on  109ms
      [+] Database tempdb should exist on  140ms
      [+] Database model should exist on  118ms
Tests completed in 2.62s
Tests Passed: 4, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


### Finding the configurations for the check

You can find the configurations that you can set for a check with `Get-DbcCheck`

In [2]:
Get-DbcCheck DatabaseExists | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : DatabaseExists
AllTags     : DatabaseExists, Database
Config      : app.sqlinstance database.exists 
Description : Tests that the databases are specified are on the instance (Note - Does not check if they are available 
              - Use DatabaseStatus for that))





You can see that you can set the configuration `database.exists` Note that we are using the `-Append` parameter to add to the configuration instead of replacing it.

In [3]:
Set-DbcConfig -Name database.exists -Value 'DBA-Admin','SQLWatch','Viennadbareports','BeardedAg_db' -Append


Name            Value                            Description                                   
----            -----                            -----------                                   
database.exists {master, msdb, tempdb, model...} The databases we expect to be on the instances




and then we can run the check again and confirm that we have all of the expected databases.

In [4]:
Invoke-DbcCheck -SqlInstance SQL2016N1 -Check DatabaseExists

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on SQL2016N1
      [+] Database master should exist on  120ms
      [+] Database msdb should exist on  103ms
      [+] Database tempdb should exist on  105ms
      [+] Database model should exist on  130ms
      [+] Database DBA-Admin should exist on  123ms
      [+] Database SQLWatch should exist on  119ms
      [+] Database Viennadbareports should exist on  109ms
      [+] Database BeardedAg_db should exist on  101ms
Tests completed in 1.18s
Tests Passed: 8, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 
